## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [3]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

if os.getenv("OPENROUTER_API_KEY"):
    print("OpenRouter API key loaded")
else:
    raise RuntimeError("OPENROUTER_API_KEY not set")


OpenRouter API key loaded


In [5]:
load_dotenv()

client = OpenAI(
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1"
)


In [6]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [5]:
print(linkedin)

   
Contact
aakarshanarya2503@gmail.com
www.linkedin.com/in/kumar-
akarshan-arya-99113a282
(LinkedIn)
Top Skills
Business Analysis
Cloud Infrastructure
Kubernetes
Kumar Akarshan Arya
Developer | UI/UX Designer | Aspiring DevOps Engineer | 1K+
LinkedIn Network | MITS Gwalior
Gwalior, Madhya Pradesh, India
Experience
Bandish
Lead Vocalist
October 2023 - Present (2 years 4 months)
Gwalior, Madhya Pradesh, India
CODTECH INTERNSHIPS
Cloud Engineer
July 2025 - August 2025 (2 months)
Hyderabad, Telangana, India
Gained hands-on experience in deploying and managing cloud infrastructure
on Aws ,assisted in monitoring, troubleshooting and optimising cloud
resources
InAmigos Foundation (IAF)
UI Designer
July 2024 - November 2024 (5 months)
India
Assisted in project planning and coordination by supporting team schedules
and task tracking 
contributed to research and reporting,preparing summaries for decision
making
CODTECH INTERNSHIPS
Java Developer
July 2024 - September 2024 (3 months)
Hyderabad, 

In [7]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [8]:
name = "Kumar Akarshan Arya"

In [9]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [12]:
system_prompt

"You are acting as Kumar Akarshan Arya. You are answering questions on Kumar Akarshan Arya's website, particularly questions related to Kumar Akarshan Arya's career, background, skills and experience. Your responsibility is to represent Kumar Akarshan Arya for interactions on the website as faithfully as possible. You are given a summary of Kumar Akarshan Arya's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nAakarshan Arya is a final-year MITS Gwalior student, developer-in-progress, and curious builder. He works with Java, Python, and agentic AI, learning by experimentation. Balancing code, music, fitness, and self-reliance, he moves forward with quiet ambition, persistence, and a growing focus on real-world impact.\n\n## LinkedIn Profile:\n\xa0 \xa0\nContact\naakarshanarya2503@gmail.com\nwww.link

In [10]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}]

    for item in history:
        # Handle tuple or list history
        if isinstance(item, (list, tuple)) and len(item) >= 2:
            user_msg = item[0]
            bot_msg = item[1]

            messages.append({"role": "user", "content": str(user_msg)})
            messages.append({"role": "assistant", "content": str(bot_msg)})

    messages.append({"role": "user", "content": str(message)})

    response = client.chat.completions.create(
        model="allenai/olmo-3.1-32b-think:free",
        messages=messages
    )

    return response.choices[0].message.content


## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [11]:
gr.ChatInterface(chat).launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [1]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel
from typing import Optional

class Evaluation(BaseModel):
    correct: bool
    verbosity: float
    feedback: Optional[str] = None


In [12]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [50]:
def evaluator_user_prompt(reply, message, history):
    return (
        "You must answer with EXACTLY one word.\n"
        "Do not explain your answer.\n"
        "Do not add punctuation.\n\n"
        "Allowed answers (not in Pig Latin):\n"
        "ACCEPTABLE\n"
        "UNACCEPTABLE\n\n"
        f"User message:\n{message}\n\n"
        f"Agent response:\n{reply}\n\n"
        "Answer:"
    )


In [14]:
import os


In [15]:
import re

def extract_json(text: str) -> str:
    match = re.search(r"\{.*\}", text, re.S)
    if not match:
        raise ValueError("No JSON object found in model output")
    return match.group(0)


In [16]:
def evaluate(reply, message, history) -> Evaluation:
    messages = [
        {"role": "system", "content": evaluator_system_prompt},
        {"role": "user", "content": evaluator_user_prompt(reply, message, history)}
    ]

    response = client.chat.completions.create(
        model="arcee-ai/trinity-mini:free",
        messages=messages,
        temperature=0
    )

    verdict = response.choices[0].message.content.strip().upper()

    return Evaluation(
        correct=verdict.startswith("ACCEPTABLE"),
        verbosity=min(len(reply.split()) / 100, 1.0),
        feedback=verdict
    )


In [36]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}]
    for item in history:
        if isinstance(item, (list, tuple)) and len(item) >= 2:
            messages.append({"role": "user", "content": str(item[0])})
            messages.append({"role": "assistant", "content": str(item[1])})

    messages.append({"role": "user", "content": str(message)})

    response = client.chat.completions.create(
        model="nousresearch/hermes-3-llama-3.1-405b:free",
        messages=messages,
        temperature=0.3
    )
    return response.choices[0].message.content


In [ ]:
history = []
reply = chat("which school did you go to?", history)
print(reply)


I attended Kendriya Vidyalaya for my schooling and completed my 12th standard in Science stream there.


In [38]:
evaluate(reply, "which school did you go to?", history)

Evaluation(correct=True, verbosity=0.16, feedback='ACCEPTABLE')

In [39]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = client.chat.completions.create(model="arcee-ai/trinity-mini:free", messages=messages)
    return response.choices[0].message.content

In [51]:
def chat(message, history):
    # 1. Build system prompt safely
    if "school" in message.lower():
        system = (
            system_prompt
            + "\n\nAll responses MUST be entirely in Pig Latin. "
            + "It is mandatory to use Pig latin strictly not english language."
        )
    else:
        system = system_prompt

    # 2. Convert history into OpenAI-style messages
    messages = [{"role": "system", "content": system}]

    for item in history:
        if isinstance(item, (list, tuple)) and len(item) >= 2:
            messages.append({"role": "user", "content": str(item[0])})
            messages.append({"role": "assistant", "content": str(item[1])})

    # 3. Add current user message
    messages.append({"role": "user", "content": str(message)})

    # 4. Call model
    response = client.chat.completions.create(
        model="arcee-ai/trinity-mini:free",
        messages=messages,
        temperature=0.3
    )

    reply = response.choices[0].message.content

    # 5. Evaluate response
    evaluation = evaluate(reply, message, history)

    # 6. Decide what to return
    if evaluation.feedback == "ACCEPTABLE":
        print("Passed evaluation – returning reply")
        return reply
    else:
        print("Failed evaluation – retrying")
        print(f"Reason: {evaluation.feedback}")
        return rerun(reply, message, history, evaluation.feedback)


In [52]:
gr.ChatInterface(chat).launch()


* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


Failed evaluation – retrying
Reason: UNACCEPTABLE
Passed evaluation – returning reply
Failed evaluation – retrying
Reason: UNACCEPTABLE
